In [3]:
import requests
from icalendar import Calendar

url = "https://canvas.auckland.ac.nz/feeds/calendars/user_q0Gw8q4wnksqiBR82PvCJjMfJoIf3pdq2BnR1vB2.ics"
data = requests.get(url).text
cal = Calendar.from_ical(data)

for event in cal.walk('VEVENT'):
    summary = event.get('summary')
    start = event.get('dtstart')
    end   = event.get('dtend')
    start_dt = start.dt if start else None
    end_dt   = end.dt if end else None
    print(summary, start_dt, end_dt)



Give feedback - DP1: Tower #1 CAD + Drawings + Feedback Template [ENGGEN 115] 2025-09-07 04:00:00+00:00 2025-09-07 04:00:00+00:00
Give feedback - DP1: Tower #1 CAD + Drawings + Feedback Template [ENGGEN 115] 2025-09-07 04:00:00+00:00 2025-09-07 04:00:00+00:00
Give feedback - DP1: Tower #1 CAD + Drawings + Feedback Template [ENGGEN 115] 2025-09-07 04:00:00+00:00 2025-09-07 04:00:00+00:00
Give feedback - DP1: Tower #1 CAD + Drawings + Feedback Template [ENGGEN 115] 2025-09-07 04:00:00+00:00 2025-09-07 04:00:00+00:00
Give feedback - DP1: Tower #1 CAD + Drawings + Feedback_M24-new [ENGGEN 115] 2025-09-07 04:00:00+00:00 2025-09-07 04:00:00+00:00
Give feedback - DP1: Tower #1 CAD + Drawings + Feedback_Th24-new [ENGGEN 115] 2025-09-07 04:00:00+00:00 2025-09-07 04:00:00+00:00
Give feedback - DP1: Tower #1 CAD + Drawings + Feedback_W46 [ENGGEN 115] 2025-09-07 04:00:00+00:00 2025-09-07 04:00:00+00:00
Give feedback as group - DP1: Tower #1 CAD + Drawings + Feedback_M24-new [ENGGEN 115] 2025-09-07

In [9]:
import requests
from icalendar import Calendar
from datetime import datetime, timezone, date
from collections import defaultdict

# --- Load your Canvas feed ---
url = "https://canvas.auckland.ac.nz/feeds/calendars/user_q0Gw8q4wnksqiBR82PvCJjMfJoIf3pdq2BnR1vB2.ics"
data = requests.get(url).text
cal = Calendar.from_ical(data)

now = datetime.now(timezone.utc)

events = []
for e in cal.walk("VEVENT"):
    start = getattr(e.get("dtstart"), "dt", None)
    end   = getattr(e.get("dtend"), "dt", None)
    title = e.get("summary", "").strip()
    if not start:
        continue

    if isinstance(start, date) and not isinstance(start, datetime):
        start = datetime.combine(start, datetime.min.time(), tzinfo=timezone.utc)

    if start <= now or start.date() == now.date():
        events.append({"title": title, "start": start, "end": end})

# --- Step 2: Group by title to detect duplicates ---
title_groups = defaultdict(list)
for ev in events:
    title_groups[ev["title"]].append(ev)

duplicates = {t: lst for t, lst in title_groups.items() if len(lst) > 1}
unique = {t: lst for t, lst in title_groups.items() if len(lst) == 1}

today = now.date()
today_dupes = []
today_unique = []
future_dupes = []
future_unique = []

for t, lst in duplicates.items():
    for ev in lst:
        if ev["start"].date() == today:
            today_dupes.append(ev)
        else:
            future_dupes.append(ev)

for t, lst in unique.items():
    for ev in lst:
        if ev["start"].date() == today:
            today_unique.append(ev)
        else:
            future_unique.append(ev)

def show(title, items):
    print(f"\n=== {title} ({len(items)}) ===")
    for ev in sorted(items, key=lambda e: e["start"]):
        print(f"[{ev['start']:%Y-%m-%d %H:%M}] {ev['title']}")

show("TODAY - Duplicates", today_dupes)
show("TODAY - Unique", today_unique)
show("FUTURE - Duplicates", future_dupes)
show("FUTURE - Unique", future_unique)



=== TODAY - Duplicates (0) ===

=== TODAY - Unique (1) ===
[2025-10-06 00:00] Lab 8 [ENGGEN 131]

=== FUTURE - Duplicates (15) ===
[2025-09-07 04:00] Give feedback - DP1: Tower #1 CAD + Drawings + Feedback Template [ENGGEN 115]
[2025-09-07 04:00] Give feedback - DP1: Tower #1 CAD + Drawings + Feedback Template [ENGGEN 115]
[2025-09-07 04:00] Give feedback - DP1: Tower #1 CAD + Drawings + Feedback Template [ENGGEN 115]
[2025-09-07 04:00] Give feedback - DP1: Tower #1 CAD + Drawings + Feedback Template [ENGGEN 115]
[2025-09-07 04:00] Give feedback - DP1: Tower #1 CAD + Drawings + Feedback_W46 [ENGGEN 115]
[2025-09-08 02:00] Give feedback as group - DP1: Tower #1 CAD + Drawings + Feedback Template [ENGGEN 115]
[2025-09-08 04:00] Give feedback - DP1: Tower #1 CAD + Drawings + Feedback Template [ENGGEN 115]
[2025-09-08 04:00] Give feedback as group - DP1: Tower #1 CAD + Drawings + Feedback Template [ENGGEN 115]
[2025-09-09 02:00] Give feedback - DP1: Tower #1 CAD + Drawings + Feedback Temp

In [11]:
import requests
from icalendar import Calendar
from datetime import datetime, timezone, date
from collections import defaultdict
from difflib import SequenceMatcher

MINIMUM_GROUPING = 6 
SIMILARITY_FILTER = 0.9

url = "https://canvas.auckland.ac.nz/feeds/calendars/user_q0Gw8q4wnksqiBR82PvCJjMfJoIf3pdq2BnR1vB2.ics"
data = requests.get(url).text
cal = Calendar.from_ical(data)

now = datetime.now(timezone.utc)

events = []
for e in cal.walk("VEVENT"):
    start = getattr(e.get("dtstart"), "dt", None)
    end   = getattr(e.get("dtend"), "dt", None)
    title = e.get("summary", "").strip()
    if not start:
        continue

    if isinstance(start, date) and not isinstance(start, datetime):
        start = datetime.combine(start, datetime.min.time(), tzinfo=timezone.utc)

    if start <= now or start.date() == now.date():
        events.append({"title": title, "start": start, "end": end})

groups = []
for ev in events:
    matched = False
    for rep_title, lst in groups:
        sim = SequenceMatcher(None, ev["title"].lower(), rep_title.lower()).ratio()
        if sim >= SIMILARITY_FILTER: 
            lst.append(ev)
            matched = True
            break
    if not matched:
        groups.append((ev["title"], [ev]))

duplicates = [(t, lst) for t, lst in groups if len(lst) > MINIMUM_GROUPING]
unique     = [(t, lst) for t, lst in groups if len(lst) == 1]

today = now.date()
today_dupes, today_unique, future_dupes, future_unique = [], [], [], []

for t, lst in duplicates:
    for ev in lst:
        if ev["start"].date() == today:
            today_dupes.append(ev)
        else:
            future_dupes.append(ev)

for t, lst in unique:
    for ev in lst:
        if ev["start"].date() == today:
            today_unique.append(ev)
        else:
            future_unique.append(ev)

def show(title, items):
    print(f"\n=== {title} ({len(items)}) ===")
    for ev in sorted(items, key=lambda e: e["start"]):
        print(f"[{ev['start']:%Y-%m-%d %H:%M}] {ev['title']}")

show("TODAY - Similar (≥90%) Duplicates", today_dupes)
show("TODAY - Unique", today_unique)
show("FUTURE - Similar (≥90%) Duplicates", future_dupes)
show("FUTURE - Unique", future_unique)



=== TODAY - Similar (≥90%) Duplicates (0) ===

=== TODAY - Unique (0) ===

=== FUTURE - Similar (≥90%) Duplicates (18) ===
[2025-09-07 04:00] Give feedback - DP1: Tower #1 CAD + Drawings + Feedback Template [ENGGEN 115]
[2025-09-07 04:00] Give feedback - DP1: Tower #1 CAD + Drawings + Feedback Template [ENGGEN 115]
[2025-09-07 04:00] Give feedback - DP1: Tower #1 CAD + Drawings + Feedback Template [ENGGEN 115]
[2025-09-07 04:00] Give feedback - DP1: Tower #1 CAD + Drawings + Feedback Template [ENGGEN 115]
[2025-09-07 04:00] Give feedback - DP1: Tower #1 CAD + Drawings + Feedback_M24-new [ENGGEN 115]
[2025-09-07 04:00] Give feedback - DP1: Tower #1 CAD + Drawings + Feedback_Th24-new [ENGGEN 115]
[2025-09-07 04:00] Give feedback - DP1: Tower #1 CAD + Drawings + Feedback_W46 [ENGGEN 115]
[2025-09-08 02:00] Give feedback - DP1: Tower #1 CAD + Drawings + Feedback [ENGGEN 115]
[2025-09-08 02:00] Give feedback as group - DP1: Tower #1 CAD + Drawings + Feedback Template [ENGGEN 115]
[2025-09-